In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
#importing train and test data
train_data = pd.read_csv(filepath_or_buffer = "/kaggle/input/titanic/train.csv")
test_data = pd.read_csv(filepath_or_buffer = "/kaggle/input/titanic/test.csv")

In [6]:
#Let's investigate the training data a little bit
train_data.head(5)

In [ ]:
# Correlation Matrix for numerical data 
cmatrix=train_data.corr()
cmatrix['Survived'].sort_values(ascending=False)

The results show Fare and Pclass have somewhat correlated. So a potential candidate for modeling

In [ ]:
 sns.barplot(data=train_data, x='Sex', y='Survived', )

In [ ]:
 sns.barplot(data=train_data, x='Embarked', y='Survived', )

In [ ]:
sns.barplot(data=train_data, x='Pclass', y='Survived', )

In [ ]:
sns.barplot(x='Sex', y='Survived', hue='Pclass', data=train_data)
plt.ylabel("Survival Rate")
plt.title("Survival as function of Pclass and Sex")
plt.show()

In [ ]:
sns.barplot(x='Embarked', y='Survived', hue='Pclass', data=train_data)
plt.ylabel("Survival Rate")
plt.title("Survival as function of Embarked Port")
plt.show()

In [ ]:
#Knowing NaNs in the Dataset is very important
print("NaN values in the DataFrame:")
train_data.isna().sum()

In [ ]:
#Knowing NaNs in the Dataset is very important
print("NaN values in the DataFrame:")
test_data.isna().sum()

In [7]:
# Conveting categorical variables to numeric to use as model inputs, missing value treatment and mean substitution

from sklearn.preprocessing import LabelEncoder

# creating instance of labelencoder
labelencoder = LabelEncoder()

# Assigning numerical values and storing in another column
train_data['Embarked'] = labelencoder.fit_transform(train_data['Embarked'].fillna('S'))
train_data['Sex'] = labelencoder.fit_transform(train_data['Sex'].fillna('female'))

test_data['Embarked'] = labelencoder.fit_transform(test_data['Embarked'].fillna('S'))
test_data['Sex'] = labelencoder.fit_transform(test_data['Sex'].fillna('female'))

#mean substitute for fare
fare_mean = train_data['Fare'].mean()
test_data['Fare'] = labelencoder.fit_transform(test_data['Fare'].fillna(fare_mean))

In [8]:
#Dropping the fields with low information values w.r.t predictor variable
train_data.drop(labels = ['PassengerId','Name','Ticket', 'Cabin','SibSp','Parch','Age'], axis = 1, inplace = True)
test_data.drop(labels = ['Name','Ticket', 'Cabin','SibSp','Parch','Age'], axis = 1, inplace = True)

In [9]:
train_data.describe()

In [10]:
X_train =train_data.iloc[:,1:5].values
Y_train =train_data.iloc[:,0].values

In [11]:
print (X_train.shape)
print (Y_train.shape)

In [12]:
X_test =test_data.iloc[:,1:5].values
print (X_test.shape)

In [13]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Decision Tree

In [28]:
from sklearn.tree import DecisionTreeClassifier

#Create classifier object with default hyperparameters
clf = DecisionTreeClassifier()  

#Fit our classifier using the training features and the training target values
clf.fit(X_train,Y_train) 

y_pred = clf.predict(X_train)

In [30]:
from sklearn.metrics import accuracy_score
ac = accuracy_score(Y_train, y_pred)

In [31]:
print(ac)

# Titanic Test File scoring

In [32]:
# Predicting the Test set results
y_pred = clf.predict(X_test)

prediction = pd.DataFrame(data = y_pred, columns = ['Survived'])
prediction.insert(0, 'PassengerId', test_data['PassengerId'])
prediction.head(5)

In [34]:
#Creating a submission
prediction.to_csv(path_or_buf = 'submission.csv', sep = ',', index = False, header = True)